In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK: Data Labeling


## Installation

Install the latest (preview) version of Vertex SDK.


In [ ]:
! pip3 install -U google-cloud-aiplatform --user

Install the Google *cloud-storage* library as well.


In [ ]:
! pip3 install google-cloud-storage

### Restart the Kernel

Once you've installed the Vertex SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.


In [ ]:
import os

if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the Vertex APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in Google Cloud Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.


In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with Vertex. Not all regions provide support for all Vertex services. For the latest support per region, see [Region support for Vertex AI services](https://cloud.google.com/vertex-ai/docs/general/locations)


In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.


In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your GCP account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an Vertex notebook and run the cell, the cell knows to skip executing the authentication steps.*


In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.


In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.


In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:


In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants


#### Import Vertex SDK

Import the Vertex SDK into our Python environment.


In [ ]:
import os
import sys
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

#### Vertex AI constants

Setup up the following constants for Vertex AI:

- `API_ENDPOINT`: The Vertex AI API service endpoint for dataset, model, job, pipeline and endpoint services.
- `API_PREDICT_ENDPOINT`: The Vertex AI API service endpoint for prediction.
- `PARENT`: The Vertex AI location root path for dataset, model and endpoint resources.


In [ ]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex AI location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### AutoML constants

Next, setup constants unique to AutoML image classification datasets and training:

- Dataset Schemas: Tells the managed dataset service which type of dataset it is.
- Data Labeling (Annotations) Schemas: Tells the managed dataset service how the data is labeled (annotated).
- Dataset Training Schemas: Tells the Vertex AI Pipelines service the task (e.g., classification) to train the model for.


In [ ]:
# Image Dataset type
IMAGE_SCHEMA = "google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml"
# Image Labeling type
IMPORT_SCHEMA_IMAGE_CLASSIFICATION = "gs://google-cloud-aiplatform/schema/dataset/ioformat/image_classification_single_label_io_format_1.0.0.yaml"
# Image labeling task
LABELING_SCHEMA_IMAGE = "gs://google-cloud-aiplatform/schema/datalabelingjob/inputs/image_classification_1.0.0.yaml"

## Clients

The Vertex SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (Vertex).

You will use several clients in this tutorial, so set them all up upfront.

- Dataset Service for managed datasets.
- Job Service for batch jobs and custom training.


In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["job"] = create_job_client()

for client in clients.items():
    print(client)

In [ ]:
import tensorflow as tf

LABELING_FILES = [
    "https://raw.githubusercontent.com/googleapis/python-aiplatform/master/samples/snippets/resources/daisy.jpg"
]

IMPORT_FILE = "gs://" + BUCKET_NAME + "/labeling.csv"
with tf.io.gfile.GFile(IMPORT_FILE, "w") as f:
    for lf in LABELING_FILES:
        ! wget {lf} | gsutil cp {lf.split("/")[-1]} gs://{BUCKET_NAME}
        f.write("gs://" + BUCKET_NAME + "/" + lf.split("/")[-1] + "\n")

In [ ]:
! gsutil cat $IMPORT_FILE

*Example output*:
```
gs://migration-ucaip-trainingaip-20210303215432/daisy.jpg
```


## Create a dataset


### [projects.locations.datasets.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.datasets/create)


#### Request


In [ ]:
DATA_SCHEMA = IMAGE_SCHEMA

dataset = {
    "display_name": "labeling_" + TIMESTAMP,
    "metadata_schema_uri": "gs://" + DATA_SCHEMA,
}

print(
    MessageToJson(
        aip.CreateDatasetRequest(parent=PARENT, dataset=dataset).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "dataset": {
    "displayName": "labeling_20210303215432",
    "metadataSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml"
  }
}
```


#### Call


In [ ]:
request = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/1165112889535627264",
  "displayName": "labeling_20210303215432",
  "metadataSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml",
  "labels": {
    "aiplatform.googleapis.com/dataset_metadata_schema": "IMAGE"
  },
  "metadata": {
    "dataItemSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/dataitem/image_1.0.0.yaml"
  }
}
```


In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### [projects.locations.datasets.import](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.datasets/import)


#### Request


In [ ]:
LABEL_SCHEMA = IMPORT_SCHEMA_IMAGE_CLASSIFICATION

import_config = {
    "gcs_source": {"uris": [IMPORT_FILE]},
    "import_schema_uri": LABEL_SCHEMA,
}

print(
    MessageToJson(
        aip.ImportDataRequest(
            name=dataset_short_id, import_configs=[import_config]
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "name": "1165112889535627264",
  "importConfigs": [
    {
      "gcsSource": {
        "uris": [
          "gs://migration-ucaip-trainingaip-20210303215432/labeling.csv"
        ]
      },
      "importSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/ioformat/image_classification_single_label_io_format_1.0.0.yaml"
    }
  ]
}
```


#### Call


In [ ]:
request = clients["dataset"].import_data(
    name=dataset_id, import_configs=[import_config]
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{}
```


### Create data labeling specialist pool


In case you do not have access to labeling services execute this section.

In [ ]:
# add client for specialist pool
clients["specialist_pool"] = aip.SpecialistPoolServiceClient(
    client_options=client_options
)

### [projects.locations.specialistPools.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.specialistPools/createe)


#### Request

In this part, you will replace [your-email-address] with your email address. This makes you the specialist and recipient of the labeling request.

In [ ]:
EMAIL = "[your-email-address]"

specialist_pool = {
    "name": "labeling_" + TIMESTAMP,  # he resource name of the SpecialistPool.
    "display_name": "labeling_" + TIMESTAMP,  # user-defined name of the SpecialistPool
    "specialist_manager_emails": [EMAIL],
}

print(
    MessageToJson(
        aip.CreateSpecialistPoolRequest(
            parent=PARENT, specialist_pool=specialist_pool
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "specialistPool": {
    "name": "labeling_20210303215432",
    "displayName": "labeling_20210303215432",
    "specialistManagerEmails": [
      "dev@fourteen33.com"
    ]
  }
}
```


#### Call


In [ ]:
request = clients["specialist_pool"].create_specialist_pool(
    parent=PARENT, specialist_pool=specialist_pool
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/specialistPools/1167839678372511744"
}
```


In [ ]:
specialist_name = result.name

specialist_id = specialist_name.split("/")[-1]

print(specialist_name)

## Create data labeling job


### [projects.locations.dataLabelingJobs.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.dataLabelingJobs/create)


In [ ]:
# create placeholder file for valid PDF file with instruction for data labeling
! echo "this is instruction" >> instruction.txt | gsutil cp instruction.txt gs://$BUCKET_NAME

#### Request


In [ ]:
LABLEING_SCHEMA = LABELING_SCHEMA_IMAGE
INSTRUCTION_FILE = "gs://" + BUCKET_NAME + "/instruction.txt"

inputs = json_format.ParseDict({"annotation_specs": ["rose"]}, Value())

data_labeling_job = {
    "display_name": "labeling_" + TIMESTAMP,
    "datasets": [dataset_id],
    "labeler_count": 1,
    "instruction_uri": INSTRUCTION_FILE,
    "inputs_schema_uri": LABLEING_SCHEMA,
    "inputs": inputs,
    "annotation_labels": {
        "aiplatform.googleapis.com/annotation_set_name": "data_labeling_job_specialist_pool"
    },
    "specialist_pools": [specialist_name],
}

print(
    MessageToJson(
        aip.CreateDataLabelingJobRequest(
            parent=PARENT, data_labeling_job=data_labeling_job
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "dataLabelingJob": {
    "displayName": "labeling_20210303215432",
    "datasets": [
      "projects/116273516712/locations/us-central1/datasets/1165112889535627264"
    ],
    "labelerCount": 1,
    "instructionUri": "gs://migration-ucaip-trainingaip-20210303215432/instruction.txt",
    "inputsSchemaUri": "gs://google-cloud-aiplatform/schema/datalabelingjob/inputs/image_classification_1.0.0.yaml",
    "inputs": {
      "annotation_specs": [
        "rose"
      ]
    },
    "annotationLabels": {
      "aiplatform.googleapis.com/annotation_set_name": "data_labeling_job_specialist_pool"
    },
    "specialistPools": [
      "projects/116273516712/locations/us-central1/specialistPools/1167839678372511744"
    ]
  }

```


#### Call


In [ ]:
request = clients["job"].create_data_labeling_job(
    parent=PARENT, data_labeling_job=data_labeling_job
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/dataLabelingJobs/3830883229125050368",
  "displayName": "labeling_20210303215432",
  "datasets": [
    "projects/116273516712/locations/us-central1/datasets/1165112889535627264"
  ],
  "labelerCount": 1,
  "instructionUri": "gs://migration-ucaip-trainingaip-20210303215432/instruction.txt",
  "inputsSchemaUri": "gs://google-cloud-aiplatform/schema/datalabelingjob/inputs/image_classification_1.0.0.yaml",
  "inputs": {
    "annotationSpecs": [
      "rose"
    ]
  },
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-03-03T21:55:31.239049Z",
  "updateTime": "2021-03-03T21:55:31.239049Z"
}
```


In [ ]:
labeling_task_name = request.name

print(labeling_task_name)

### [projects.locations.dataLabelingJobs.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.dataLabelingJobs/get)


#### Call


In [ ]:
request = clients["job"].get_data_labeling_job(name=labeling_task_name)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/dataLabelingJobs/3830883229125050368",
  "displayName": "labeling_20210303215432",
  "datasets": [
    "projects/116273516712/locations/us-central1/datasets/1165112889535627264"
  ],
  "labelerCount": 1,
  "instructionUri": "gs://migration-ucaip-trainingaip-20210303215432/instruction.txt",
  "inputsSchemaUri": "gs://google-cloud-aiplatform/schema/datalabelingjob/inputs/image_classification_1.0.0.yaml",
  "inputs": {
    "annotationSpecs": [
      "rose"
    ]
  },
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-03-03T21:55:31.239049Z",
  "updateTime": "2021-03-03T21:55:31.239049Z",
  "specialistPools": [
    "projects/116273516712/locations/us-central1/specialistPools/1167839678372511744"
  ]
}
```


### [projects.locations.dataLabelingJobs.cancel](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.dataLabelingJobs/cancel)


#### Call


In [ ]:
request = clients["job"].cancel_data_labeling_job(name=labeling_task_name)

#### Response


In [ ]:
print(request)

*Example output*:
```
None
```


In [ ]:
while True:
    response = clients["job"].get_data_labeling_job(name=labeling_task_name)
    if response.state == aip.JobState.JOB_STATE_CANCELLED:
        print("Labeling job CANCELED")
        break
    else:
        print("Canceling labeling job:", response.state)
        time.sleep(60)

# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.


In [ ]:
delete_dataset = True
delete_job = True
delete_specialist_pool = True
delete_bucket = True

# Delete the dataset using the Vertex AI fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the labeling job using the Vertex AI fully qualified identifier for the dataset
try:
    if delete_job:
        request = clients["job"].delete_data_labeling_job(name=labeling_task_name)
except Exception as e:
    print(e)

# Delete the specialist pool using the Vertex AI fully qualified identifier for the dataset
try:
    if delete_specialist_pool:
        clients["specialist_pool"].delete_specialist_pool(name=specialist_name)
except Exception as e:
    print(e)


if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME